In [15]:
import os
import csv
import time
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
import os

import warnings
warnings.filterwarnings("ignore")

In [17]:
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

def get_category_txt():
    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome()

    # Navigate to the website
    driver.get('https://auto.drom.ru/')

    # Find the button using its attributes
    button = driver.find_element(By.XPATH, '//button[@data-ftid="component_cars-list_expand-control"]')
    button.click()
    time.sleep(3)

    # Save the page source to a file
    if not os.path.exists("auto_html"):
        os.mkdir("auto_html")

    with open("auto_html/auto_drom_ru.html", "w", encoding='utf_8') as f:
        f.write(driver.page_source)
    
    # Close the driver
    driver.quit()
    

    # Открываем полученный файл (парсим ссылки, если более 2000, то берем ссылки брендов, то есть каждую модель)
    with open("auto_html/auto_drom_ru.html", encoding='utf_8') as file:
        src = file.read()
        
    soup = BeautifulSoup(src, 'lxml')
    cat_tags = soup.findAll('div', class_='css-u4n5gw e4ojbx41')
    
    urls_list = []
    for tag in cat_tags:
        if tag.find('span', class_='css-3pvdar e4ojbx45') == None:
            pass
        
        elif int(tag.find('span', class_='css-3pvdar e4ojbx45').text.strip()) > 2000:
            cat_url = tag.find('a', class_='css-1q66we5 e4ojbx43').get('href')
            
            # Get the HTML content of the page
            r = requests.get(cat_url, headers=headers)
            soup_inside = BeautifulSoup(r.text, 'lxml')
            if soup_inside.find('div', class_='css-atyutb e1cj4jfp2') == None:
                inside_cat_list = soup_inside.findAll('div', class_='css-m7q1zs e4ojbx42')
                
                # Убираем тракторы, яхты и прочее из подкатегорий
                for element in inside_cat_list:
                    if element.find('span', class_='css-3pvdar e4ojbx45') != None:
                        car_tag = element.find('a', class_='css-1q66we5 e4ojbx43').get('href')
                        urls_list.append(car_tag)
            
            else:
                driver = webdriver.Chrome()

                # Navigate to the website
                driver.get(cat_url)

                # Find the button using its attributes and click it
                button = driver.find_element(By.XPATH, '//button[@data-ftid="component_cars-list_expand-control"]')
                button.click()
                time.sleep(1)

                # Parse the HTML content using BeautifulSoup
                soup_inside_unhide = BeautifulSoup(driver.page_source, 'lxml')
                inside_cat_list = soup_inside_unhide.findAll('div', class_='css-m7q1zs e4ojbx42')
                
                # Убираем тракторы, яхты и прочее из подкатегорий
                for element in inside_cat_list:
                    if element.find('span', class_='css-3pvdar e4ojbx45') != None:
                        car_tag = element.find('a', class_='css-1q66we5 e4ojbx43').get('href')
                        urls_list.append(car_tag)
                
                # Close the driver
                driver.quit()
                
        else:
            cat_url = tag.find('a', class_='css-1q66we5 e4ojbx43').get('href')
            if cat_url != 'https://auto.drom.ru/other/':
                urls_list.append(cat_url)

    with open("auto_html\category_urls.txt", "w") as file:
        file.write('\n'.join(urls_list))
    
    return print('[INFO] Список категорий получен, ссылки сохранены в файл: auto_html\category_urls.txt')


def get_list_categories(file_path):
    with open("auto_html\category_urls.txt") as file:
        categories_list = [line.strip() for line in file.readlines()]    
    
    return categories_list


def create_csv(ver):
    # Если директория отсутствует, то создаем ее:
    if not os.path.exists("csv"):
        os.mkdir("csv")
    
    # фиксируем текущую дату
    curr_date = datetime.now().strftime("%Y-%m-%d")
    
    # create csv
    with open(f"csv/{curr_date}_data_{ver}.csv", "w", newline="") as file:
        writer = csv.writer(file, delimiter=";")
        row = [
            'curr_date', 'name', 'bodyType', 'brand_name', 'color', 'fuelType',
            'productionDate', 'modelDate', 'numberOfDoors',
            'vehicleConfiguration', 'vehicleTransmission',
            'vehicleEngine_name', 'vehicleEngine_engineDisplacement',
            'vehicleEngine_enginePower', 'offers_price', 'offers_priceCurrency',
            'offers_url',
            'card_charact', 'card_price_disc', 'card_price_mark',
            'card_repair', 'card_noDocs', 'card_probegCat', 'card_probegCat1'
        ]
        writer.writerow(row)
        
    return curr_date


def collect_data(ver, max_pages, category):
    # фиксируем текущую дату
    curr_date = datetime.now().strftime("%Y-%m-%d")
    
    # Парсер
    data=[]
    for page in range(1, max_pages + 1):
        time.sleep(1)
        
        url_page = f"{category}page{page}/?unsold=1"
        r = requests.get(url_page, headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')
        
        if soup.find('a', class_='css-4zflqt e1huvdhj1') == None:
            break
        else:
            soup_html = BeautifulSoup(r.text, 'html.parser')
            script_tags = soup.find_all('script', type='application/ld+json')

            price_tag = soup_html.find('script', type='application/ld+json')
            price_dict = json.loads(price_tag.string)


            for i, script_tag in enumerate(script_tags):
                car_dict = json.loads(script_tag.string)

                if car_dict.get('@type') == 'Car':
                    name = car_dict.get('name', '')
                    bodyType = car_dict.get('bodyType', '')
                    brand_name = car_dict['brand'].get('name', '')
                    color = car_dict.get('color', '')
                    fuelType = car_dict.get('fuelType', '')
                    productionDate = car_dict.get('productionDate', '')
                    modelDate = car_dict.get('modelDate', '')
                    numberOfDoors = car_dict.get('numberOfDoors', '')

                    vehicleConfiguration = car_dict.get('vehicleConfiguration', '')
                    vehicleTransmission = car_dict.get('vehicleTransmission', '')

                    vehicleEngine = car_dict.get('vehicleEngine', 'NaN')
                    if vehicleEngine != 'NaN':
                        vehicleEngine_name = vehicleEngine.get('name', '')
                        vehicleEngine_engineDisplacement = vehicleEngine.get('engineDisplacement', '')
                        vehicleEngine_enginePower = vehicleEngine.get('enginePower', '')
                    else:
                        vehicleEngine_name = ''
                        vehicleEngine_engineDisplacement = ''
                        vehicleEngine_enginePower = ''

#                     all_offers = car_dict.get('offers', 'NaN')
#                     if all_offers != 'NaN':
#                         offer_lowPrice = all_offers.get('lowPrice', '')
#                         offer_highPrice = all_offers.get('highPrice', '')
#                         offer_offerCount = all_offers.get('offerCount', '')
#                     else:
#                         offer_lowPrice = ''
#                         offer_highPrice = ''
#                         offer_offerCount = ''


                    # info from price JSON
                    offers = price_dict['offers']['offers'][i - 1]
                    offers_price = offers.get('price', '')
                    offers_priceCurrency = offers.get('priceCurrency', '')
#                     offers_availability = offers.get('availability', '')
                    offers_url = offers.get('url', '')
#                     offers_priceValidUntil = offers.get('priceValidUntil', '')

                    # info from car card
                    # soup_ = BeautifulSoup(src, "lxml")
                    item_card = soup.findAll('a', class_='css-4zflqt e1huvdhj1')[i - 1]
                    try:
                        card_charact = item_card.find("div", class_="css-1fe6w6s e162wx9x0").text.strip().replace(' км', '')
                    except:
                        card_charact = ''

                    try:
                        card_price_disc = item_card.find("div", class_="css-4h3sx1 e162wx9x0").text.strip().replace('₽', '')
                    except:
                        card_price_disc = ''

                    try:
                        card_price_mark = item_card.find("div", class_="css-1femo5v evjskuu0").text.strip().replace(' ', '_')
                    except:
                        card_price_mark = ''

                    try:
                        card_repair = item_card.find('div', {'data-ftid': 'bull_label_broken'}).get('title', 'repaired')
                    except:
                        card_repair = ''

                    try:
                        card_noDocs = item_card.find('div', {'data-ftid': 'bull_label_nodocs'}).get('title', 'noDocs')
                    except:
                        card_noDocs = ''

#                     try:
#                         card_from_owner = item_card.find('div', class_='css-19fru5w ejipaoe0').text.strip().replace(' ', '_')
#                     except:
#                         card_from_owner = ''

                    try:
                        card_probegCat = item_card.find('div', class_='css-1jdyedu ejipaoe0').text.strip().replace(' ', '_')
                    except:
                        card_probegCat = ''
                        
                    try:
                        card_probegCat1 = item_card.find('div', class_='css-4qlhiu ejipaoe0').get('title', 'NaN')
                    except:
                        card_probegCat1 = ''


                    # print(i, card_from_owner, ' - ' , card_probegCat)

                    data.append(
                        {
                            'curr_date': curr_date, 
                            'name': name,
                            'bodyType': bodyType,
                            'brand_name': brand_name,
                            'color': color,
                            'fuelType': fuelType,
                            'productionDate': productionDate,
                            'modelDate': modelDate,
                            'numberOfDoors': numberOfDoors,
                            'vehicleConfiguration': vehicleConfiguration,
                            'vehicleTransmission': vehicleTransmission,
                            'vehicleEngine_name': vehicleEngine_name,
                            'vehicleEngine_engineDisplacement': vehicleEngine_engineDisplacement,
                            'vehicleEngine_enginePower': vehicleEngine_enginePower,
                            # 'offer_lowPrice': offer_lowPrice,
                            # 'offer_highPrice': offer_highPrice,
                            # 'offer_offerCount': offer_offerCount,
                            'offers_price': offers_price,
                            'offers_priceCurrency': offers_priceCurrency,
                            # 'offers_availability': offers_availability,
                            'offers_url': offers_url,
                            # 'offers_priceValidUntil': offers_priceValidUntil,
                            'card_charact' : card_charact,
                            'card_price_disc': card_price_disc,
                            'card_price_mark': card_price_mark,
                            'card_repair': card_repair,
                            'card_noDocs': card_noDocs,
                            # 'card_from_owner': card_from_owner,
                            'card_probegCat': card_probegCat,
                            'card_probegCat1': card_probegCat1
                        }
                    )

                    with open(f"csv/{curr_date}_data_{ver}.csv", "a", newline="") as file:
                        writer = csv.writer(file, delimiter=";")
                        writer.writerow((curr_date, name, bodyType, brand_name, color, fuelType, productionDate, modelDate, numberOfDoors, vehicleConfiguration, vehicleTransmission, vehicleEngine_name, vehicleEngine_engineDisplacement, vehicleEngine_enginePower, offers_price, offers_priceCurrency, offers_url, card_charact, card_price_disc, card_price_mark, card_repair, card_noDocs, card_probegCat, card_probegCat1))
                    
        # print(f'[INFO] Обработана страница: {url_page}')

                            
def main():
    # Обновить лист категорий (Selenium)
    # get_category_txt()

    # Парсер
    cat_list = get_list_categories('auto_html\category_urls.txt')
    
    for i, cat in enumerate(cat_list):
        try_ = 5
        save_threshold = 300
        file_version = f'{try_}_{i // save_threshold}'
        
        if i % save_threshold == 0:
            create_csv(ver=file_version)
            print('\n', f'Создан новый файл: {file_version}')
            collect_data(ver=file_version, max_pages=100, category=cat)
            print(f'[{i + 1}/{len(cat_list)}] Section DONE: {cat}')
        else:
            collect_data(ver=file_version, max_pages=100, category=cat)
            print(f'[{i + 1}/{len(cat_list)}] Section DONE: {cat}')


if __name__ == "__main__":
    main()


 Создан новый файл: 4_0
[1/419] Section DONE: https://auto.drom.ru/audi/sq5_sportback/
[2/419] Section DONE: https://auto.drom.ru/chevrolet/van-model/
[3/419] Section DONE: https://auto.drom.ru/eagle/
[4/419] Section DONE: https://auto.drom.ru/geely/vision_x3/
[5/419] Section DONE: https://auto.drom.ru/honda/that~s/
[6/419] Section DONE: https://auto.drom.ru/hyundai/ix20/
[7/419] Section DONE: https://auto.drom.ru/infiniti/m56/
[8/419] Section DONE: https://auto.drom.ru/jac/
[9/419] Section DONE: https://auto.drom.ru/kia/cee~d/
[10/419] Section DONE: https://auto.drom.ru/kia/pro_cee~d/
[11/419] Section DONE: https://auto.drom.ru/lexus/gs400/
[12/419] Section DONE: https://auto.drom.ru/mitsubishi/expo/
[13/419] Section DONE: https://auto.drom.ru/mitsubishi/sapporo/
[14/419] Section DONE: https://auto.drom.ru/nissan/pick-up/
[15/419] Section DONE: https://auto.drom.ru/nissan/r~nessa/
[16/419] Section DONE: https://auto.drom.ru/peugeot/304/
[17/419] Section DONE: https://auto.drom.ru/sko

[137/419] Section DONE: https://auto.drom.ru/toyota/crown/
[138/419] Section DONE: https://auto.drom.ru/toyota/crown_kluger/
[139/419] Section DONE: https://auto.drom.ru/toyota/crown_majesta/
[140/419] Section DONE: https://auto.drom.ru/toyota/curren/
[141/419] Section DONE: https://auto.drom.ru/toyota/cynos/
[142/419] Section DONE: https://auto.drom.ru/toyota/deliboy/
[143/419] Section DONE: https://auto.drom.ru/toyota/duet/
[144/419] Section DONE: https://auto.drom.ru/toyota/echo/
[145/419] Section DONE: https://auto.drom.ru/toyota/esquire/
[146/419] Section DONE: https://auto.drom.ru/toyota/estima/
[147/419] Section DONE: https://auto.drom.ru/toyota/estima_emina/
[148/419] Section DONE: https://auto.drom.ru/toyota/estima_lucida/
[149/419] Section DONE: https://auto.drom.ru/toyota/fj_cruiser/
[150/419] Section DONE: https://auto.drom.ru/toyota/fortuner/
[151/419] Section DONE: https://auto.drom.ru/toyota/frontlander/
[152/419] Section DONE: https://auto.drom.ru/toyota/funcargo/
[153/

[270/419] Section DONE: https://auto.drom.ru/toyota/xa/
[271/419] Section DONE: https://auto.drom.ru/toyota/yaris/
[272/419] Section DONE: https://auto.drom.ru/toyota/yaris_cross/
[273/419] Section DONE: https://auto.drom.ru/toyota/yaris_verso/
[274/419] Section DONE: https://auto.drom.ru/volkswagen/amarok/
[275/419] Section DONE: https://auto.drom.ru/volkswagen/arteon/
[276/419] Section DONE: https://auto.drom.ru/volkswagen/atlas/
[277/419] Section DONE: https://auto.drom.ru/volkswagen/atlas_cross_sport/
[278/419] Section DONE: https://auto.drom.ru/volkswagen/beetle/
[279/419] Section DONE: https://auto.drom.ru/volkswagen/bora/
[280/419] Section DONE: https://auto.drom.ru/volkswagen/caddy/
[281/419] Section DONE: https://auto.drom.ru/volkswagen/california/
[282/419] Section DONE: https://auto.drom.ru/volkswagen/caravelle/
[283/419] Section DONE: https://auto.drom.ru/volkswagen/eos/
[284/419] Section DONE: https://auto.drom.ru/volkswagen/fox/
[285/419] Section DONE: https://auto.drom.r

ConnectionError: HTTPSConnectionPool(host='auto.drom.ru', port=443): Max retries exceeded with url: /volkswagen/golf_plus/page10/?unsold=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000167F0E40850>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

In [14]:
# Get data from folder path and create merged csv
def get_files_list(folder_path):
    file_names = os.listdir(folder_path)
    files_list = []
    for file_name in file_names:
        files_list.append(file_name)
    return files_list


def merge_csv_files(folder_path, file_list):
    dfs = []
    for file in file_list:
        df = pd.read_csv(f"{folder_path}{file}", sep=";", encoding='cp1251')
        dfs.append(df)
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df = merged_df.drop_duplicates(keep='first').reset_index(drop=True)
    return merged_df


# Execute code
folder_path = "C:/Users/Александр/OneDrive/Python/Python_parse/auto_drom/csv/2024-03-28/"
# merge_csv_files(folder_path, get_files_list(folder_path))

merged_df = merge_csv_files(folder_path, get_files_list(folder_path))
merged_df.to_csv(f"{folder_path}autodrom_unprocessed_df.csv",
                 sep=';',
                 encoding='cp1251')